<a href="https://colab.research.google.com/github/frontloss/RAG-System-for-News-Feeds/blob/main/Retrieval_Augmented_Generation_(RAG)_System_for_News_Feeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git config --global user.email "abhinavnitr123@gmail.com"
!git config --global user.name "Abhinav.K"
!echo "# RAG-system-for-News-Feeds" >> README.md
!git init
!git add README.md
!git commit -m "RAG LLM code"
!git branch -M main
!git remote add origin https://github.com/frontloss/RAG-system-for-News-Feeds.git
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
[main 6e7d407] RAG LLM code
 1 file changed, 1 insertion(+)
error: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!pip install feedparser

In [ ]:
!pip install replicate

In [ ]:
import feedparser
import re
import os
os.environ["REPLICATE_API_TOKEN"] = "r8_6e1GCpZqGq27sSMV6umEFjRkHk3QWaG0DsZa5"
import replicate

In [ ]:
# RSS feed URL for the New York Times Science section
main_url = "https://rss.nytimes.com/services/xml/rss/nyt"
topics = ['/Business.xml','/Technology.xml','/Sports.xml','/Science.xml','/Health.xml','/Arts.xml','/Style.xml']
# Parse the RSS feed
for topic in topics:
    feed = feedparser.parse(main_url + topic)
    # Extract and print the combined title and description of each entry in the feed
    for entry in feed.entries:
        content = f"{entry.title}\n{entry.description}\n\n"
        with open(topic[1:-4]+'.txt','a') as f:
            f.write(content)


In [ ]:
questions = ['What are the convincing arguments to ease trade restrictions on Marijuana?',
             'What kind of formal suits does Elon Musk wear for company meetings?']

In [ ]:
classify_prompt = '''Classify the above question into one of the following categories:

1. Business
2. Technology
3. Sports
4. Science
5. Health
6. Arts
7. Style'''

def generate_text(prompt,system_prompt):
    api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])
    result = ''
    for event in api.stream(
        "meta/llama-2-70b-chat",
        input={
            "debug": False,
            "top_k": 50,
            "top_p": 1,
            "prompt": prompt,
            "temperature": 0.02,
            "system_prompt": system_prompt,
            "max_new_tokens": 500,
            "min_new_tokens": -1
        },
    ):
        result+=str(event)+' '
    return result

In [ ]:
system_prompt = "Don't give me any explanation. I just need the topic category as the answer."
responses = []
for ques in questions:
    prompt = ques + '\n' + classify_prompt
    response = generate_text(prompt, system_prompt)
    responses.append(response)

In [ ]:
categories = []
for response in responses:
    pattern = r'\b\d+[\. \:]\s*[A-Za-z]{1,}\b'
    match_status = re.search(pattern, response)
    if match_status:
        result = match_status.group()
        result = re.findall('[A-Za-z]{1,}',result)[0]
        categories.append(result)

In [ ]:
responses

[' The  above question can  be classified  into category  5: Health .  ',
 '\n The  above question can  be classified  into category  7: Style .  ']

In [ ]:
categories

['Health', 'Style']

In [ ]:
def get_master_prompt(category_text, question):
    master_prompt = "text:" + "\n" + category_text + "\n" + "question:" + "\n" + question + "Use the above given portion of the text to see if any part is relevant to answer the above question. \n Return the relevant text verbatim."
    return master_prompt

system_prompt = "If there are no relevant text. reply no relevant news article found .Also, you are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
answers = []
for category,question in zip(categories,questions):
    category_text = ''
    with open(category+'.txt') as f:
        category_text += f.read()
    master_prompt = get_master_prompt(category_text, question)
    answer = generate_text(master_prompt, system_prompt)
    answers.append(answer)

In [ ]:
answers

['\n\n According  to the  text,  federal scientists have  recommended eas ing restrictions  on marijuana . The  text states , " Federal Scient ists Re commend  Easing  Restrictions on  Mariju ana"  and " In newly disclosed  documents,  federal research ers find  that cannabis  may have  medical uses  and is  less likely to cause  harm than  drugs  like hero in." This suggests  that there  are convin cing arguments  to ease trade restrictions  on mar ijuana  based on  its potential medical benefits  and lower  risk of  harm compared  to other drugs .   ',
 "\n\n\n No  relevant text  was found  in the  provided portion  of the text to  answer the  question about  Elon  Musk 's formal su its for  company meet ings.  The text only discuss es fashion , watch es,  and lux ury brands,  but does  not mention  anything about  Elon Musk  or his  attire .\n\nThere fore,  I cannot  provide a  relevant answer to this  question based  on the  given text .   "]

In [ ]:
i = 0
for answer in answers:
    answer = answer.replace('\n','')
    answers[i] = answer
    i+=1

In [ ]:
i = 1
for question, answer in zip(questions, answers):
    print('Question {0}:'.format(i))
    print(question)
    print('\n')
    print('Answer {0}:'.format(i))
    print(answer)
    i+=1
    print('\n')

Question 1:
What are the convincing arguments to ease trade restrictions on Marijuana?


Answer 1:
 According  to the  text,  federal scientists have  recommended eas ing restrictions  on marijuana . The  text states , " Federal Scient ists Re commend  Easing  Restrictions on  Mariju ana"  and " In newly disclosed  documents,  federal research ers find  that cannabis  may have  medical uses  and is  less likely to cause  harm than  drugs  like hero in." This suggests  that there  are convin cing arguments  to ease trade restrictions  on mar ijuana  based on  its potential medical benefits  and lower  risk of  harm compared  to other drugs .   


Question 2:
What kind of formal suits does Elon Musk wear for company meetings?


Answer 2:
 No  relevant text  was found  in the  provided portion  of the text to  answer the  question about  Elon  Musk 's formal su its for  company meet ings.  The text only discuss es fashion , watch es,  and lux ury brands,  but does  not mention  anything a